# Basic exemple

In [70]:
import numpy as np
from roseau.load_flow import *

Network creation

In [71]:
ground = Ground()
pref = PotentialRef(ground)  # Set the ground to 0

# Create a voltage source
un = 400 / np.sqrt(3)
voltages = [un, un * np.exp(-2j * np.pi / 3), un * np.exp(2j * np.pi / 3)]
vs = VoltageSource(id="vs", n=4, ground=ground, voltages=voltages)

# Add a new LV bus and a load
load_bus = Bus(id="bus", n=4)
load = PowerLoad(id="load", n=4, bus=load_bus, s=[10e3, 10e3, 10e3])

# Add a LV line between the voltage source and the bus
lc = LineCharacteristics("z", z_line=0.1 * np.eye(4, dtype=complex))
line = SimplifiedLine(id="line", n=4, bus1=vs, bus2=load_bus, line_characteristics=lc, length=2.0)

In [72]:
# Create the electrical network
en = ElectricalNetwork.from_element(vs)

Load flow solving

In [73]:
# Solve the load flow
en.solve_load_flow(auth=("username", "dev password"))

2

Results

In [74]:
load_bus.potentials

Magnitude,[ 2.21928183e+02+5.44506370e-21j -1.10964092e+02-1.92195445e+02j -1.10964092e+02+1.92195445e+02j 2.35450397e-15+1.64993039e-19j]
Units,volt


In [75]:
line.currents

(array([ 4.50596216e+01-2.72253185e-20j, -2.25298108e+01-3.90227770e+01j,
        -2.25298108e+01+3.90227770e+01j, -1.17725198e-14-8.24965196e-19j]) <Unit('ampere')>,
 array([-4.50596216e+01+2.72253185e-20j,  2.25298108e+01+3.90227770e+01j,
         2.25298108e+01-3.90227770e+01j,  1.17725198e-14+8.24965196e-19j]) <Unit('ampere')>)

In [76]:
buses_results, branches_results, loads_results = en.results
buses_results

potential
bus_id phase                            
vs     a      2.309401e+02+0.000000e+00j
       b     -1.154701e+02-2.000000e+02j
       c     -1.154701e+02+2.000000e+02j
       n      0.000000e+00+0.000000e+00j
bus    a      2.219282e+02+5.445064e-21j
       b     -1.109641e+02-1.921954e+02j
       c     -1.109641e+02+1.921954e+02j
       n      2.354504e-15+1.649930e-19j

In [77]:
branches_results

current1                    current2
branch_id phase                                                        
line      a      4.505962e+01-2.722532e-20j -4.505962e+01+2.722532e-20j
          b     -2.252981e+01-3.902278e+01j  2.252981e+01+3.902278e+01j
          c     -2.252981e+01+3.902278e+01j  2.252981e+01-3.902278e+01j
          n     -1.177252e-14-8.249652e-19j  1.177252e-14+8.249652e-19j

Network update

In [78]:
# Change the load to an unbalanced one and rerun the load flow
load.update_powers([15e3, 0, 0])
en.solve_load_flow(auth=("username", "dev password"))
load_bus.potentials

Magnitude,[ 216.02252269 +0.j -115.47005384-200.j -115.47005384+200.j 14.91758499 +0.j]
Units,volt
